In [1]:
from pynq.overlays.base import BaseOverlay

base = BaseOverlay('base.bit')

In [2]:
base.init_rf_clks()

In [ ]:
#!/usr/bin/env python
"""
"""
import redis
import numpy as np
import json
import time

def set_transmitter_channel(channel, enable, gain, frequency):
    channel.control.enable = enable
    channel.control.gain = gain
    channel.dac_block.MixerSettings['Freq'] = frequency
    

def run_stream():
    """Set ups a redis connection and updates data to the stream."""
    stream_name="bu_rfsoc"
    r = redis.Redis(host='155.41.48.219', port=6379, db=0,password='Qsh4r9.VlMj5_HrvY#0f36')
    stream_name="bu_rfsoc"
    
    number_samples = 1024
    
    channel = 0
    window = np.array(np.blackman(number_samples)[:]) #take window from data
    print(base.radio.transmitter.channel)
    fs=base.radio.transmitter.channel[channel].dac_block.BlockStatus['SamplingFreq']*1e9
    print(f'{fs}')
        
    r.sadd("active_streams", stream_name)
    
    if channel == 1:
        set_transmitter_channel(base.radio.transmitter.channel[1], True, 0.8,  600)

    fc=base.radio.transmitter.channel[channel].dac_block.MixerSettings['Freq']*1e6
    print(f'{fc}')

    r.delete(f"stream:{stream_name}")

    try:
        while True:
            cdata = base.radio.receiver.channel[channel].transfer(number_samples) #get complex data from ADCs
            wdata=cdata*window
            fdata=abs(np.fft.fftshift(np.fft.fft(wdata)))

            y_max = max(fdata)
            y_min = min(fdata)
            
            metadata = {
                'sfreq':    fs,
                'y_max':    y_max,
                'y_min':    y_min,
                'n_samples':number_samples,
                'cfreq':    fc,
                'channel':  channel
            }
            r.hset(f"metadata:{stream_name}", mapping=metadata)

            x_json = json.dumps(fdata.tolist())
            r.xadd(f"stream:{stream_name}", {'data': x_json}, maxlen=1000000)
                        
    finally:
        print("ENTER FINALLY")
        r.srem("active_streams", stream_name)
        r.delete(f"metadata:{stream_name}")
        r.delete(f"stream:{stream_name}")
        
if __name__ == '__main__':
    run_stream()

[<rfsystem.hierarchies.DacChannel object at 0xffff81d7b6d0>, <rfsystem.hierarchies.DacChannel object at 0xffff81d516a0>]
4096000000.0
1024000000.0
